This Python notebook contains the code I wrote for my UGRC problem on 'DNA-based computing for the Subset Product Problem'.

In [1]:
import random
import math
pairings = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}    # Setting the rules for complementary base pairing
K = 16    # Length of each DNA sequence generated by us

In [2]:
def Append(T, x):
    """
    Appends a DNA strand to the 3' end of every DNA strand in a given test tube.

    Parameters:
    T (list of str): A list (test tube) of DNA strands.
    x (str): The DNA strand to append.

    Returns:
    list of str: A new list (test tube) with x appended to the 3' end of every strand in T.
    """
    return [s + x for s in T]

def Append_head(T, x):
    """
    Appends a DNA strand to the 5' end of every DNA strand in a given test tube.

    Parameters:
    T (list of str): A list (test tube) of DNA strands.
    x (str): The DNA strand to append.

    Returns:
    list of str: A new list (test tube) with x appended to the 5' end of every strand in T.
    """
    return [x + s for s in T]

def Merge(*args):
    """
    Merges contents of multiple test tubes into one without changing any individual DNA strands.

    Parameters:
    *args (list of list of str): Variable number of lists representing variable number of test tubes containing DNA.

    Returns:
    list of str: A merged list of strands containing all DNA strands from the input test tubes.
    """
    merged_list = []
    for lst in args:
        merged_list.extend(lst)
    return merged_list

def Amplify(T):
    """
    Takes in a test tube containing DNA and produces a duplicate.

    Parameters:
    T (list of str): Represents a list of DNA strands in a test tube called T

    Returns:
    tuple of list of str: Two identical lists representing the original and its duplicate.
    """
    return T, T.copy()

def Detect(T):
    """
    Check if a particular test tube contains at least 1 DNA strand.

    Parameters:
    T (list of str): A list of DNA strands in a test tube.

    Returns:
    bool: True if T contains at least one DNA strand, False otherwise.
    """
    return bool(T)

def Extract(T, x):
    """
    Extracts DNA strands from a test tube that contain a specific sub-sequence.

    Parameters:
    T (list of str): A list of DNA strands in a test tube.
    x (str): The DNA sub-sequence to look for.

    Returns:
    list of str: A list of strands from T that contain x.
    """
    return [string for string in T if x in string]

def Remove(T, s):
    """
    Removes DNA strands from a test tube that contain a specific sub-sequence.

    Parameters:
    T (list of str): A list of DNA strands in a test tube.
    s (str): The DNA sub-sequence to remove.

    Returns:
    list of str: A list of strands from T that do not contain s.
    """
    return [st for st in T if s not in st]

def find_complement(strand):
    """
    Returns the DNA complement a given strand.

    Parameters:
    strand (str): A DNA strand.

    Returns:
    str: Its complementary DNA strand.
    """
    output = ""
    for nuc in strand: output += pairings[nuc]
    return output


In [3]:
def Init(n1, n2):
    """
    PCR simulation.
    Starts with an empty test tube.
    In each iteration, focuses on one element from input list L.
    Duplicates the contents of the test tube.
    To one copy, append a sequence indicating the presence of the current element. To the other copy, append a sequence indicating the absence of the current element.
    After each iteration, it merges the contents of the tubes.

    Parameters:
    n1 (int): The starting index of the range of elements.
    n2 (int): The ending index of the range of elements.

    Returns:
    list of str: A list of DNA strands representing all possible subsets of elements in the range [n1, n2].
    """
    T0 = ['']    # Start with an empty test tube
    for i in range(n1, n2):
        T1, T2 = Amplify(T0)       # At every iteration, duplicate the contents of the test tube to return 2 new test tubes T1 and T2
        T1 = Append(T1, x[i][0])   # Append a strand indicating PRESENCE of element i, to every strand in T1
        T2 = Append(T2, x[i][1])   # Append a strand indicating ABSENCE of element i, to every strand in T1
        T0 = Merge(T1, T2)         # Merge the contents of T1 and T2
    return T0


In [4]:
y1 = [ ''.join([ random.choice(list(pairings.keys())) for _ in range(K) ]) for _ in range(5000) ]
y0 = [ find_complement(v) for v in y1 ]

# y1 and y0 are lists of DNA sequences that will be used to represent bits in our calculations
# y1[i] indicates that bit i = 1
# y0[i] indicates that bit i = 0

In [5]:
def Value(T0, n):
    """
    Processes a test tube T0 to append sequences representing binary values of the elements present in the subset corresp. to each strand

    For each iteration, it extracts subsets of DNA strands from T0 that either contain or do not contain a specific element 'm'.
    It then appends sequences to these subsets representing binary values, depending on whether the element is present or absent.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube.
    n: number of elements in L

    Returns:
    T0 (list of str): The modified list of DNA strands after appending the binary value sequences.
    """
    for m in range(n):
        T1 = Extract(T0, x[m][0])
        T2 = Extract(T0, x[m][1])

        # For all subsets containing element m, append 16 sequences representing the bits in m
        if(Detect(T1)):
            for j in range(16):
                T1 = Append_head(T1, c[m][j])

        # For all subsets not containing element m, append 16 sequences representing 0 for all 16 bits
        if(Detect(T2)):
            for j in range(16):
                T2 = Append_head(T2, y0[16*m+j])

        T0 = Merge(T1, T2)  # Merge T1 and T2 to give T0
    return T0


In [6]:
def InitialSum(T0, n):
    """
    Initializes the sum to 0 for each subset in a test tube.

    In each iteration, the appended sequence represents a binary zero. The sequences representing binary zeros are selected from the global list 'y0'.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube.
    n: number of elements in L

    Returns:
    list of str: The modified list of DNA strands after initializing the sum to 0 for each subset.
    """
    for i in range(32):
        T0 = Append_head(T0, y0[16*n + i])
    return T0


In [7]:
def OneBitAdder(T0, f, h, g):

  """

    This function simulates a one-bit full adder in parallel across multiple DNA strands.
    It adds the bits at three specified bit positions (f, g, and h) in each DNA strand of the test tube T0.
    Based on these bits, it appends corresponding sum and carry bits to each DNA strand.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube.
    f, h, g (int): The bit positions to be added.

    Returns:
    list of str: The modified list of DNA strands after performing the parallel one-bit addition.

    Function works by first extracting subsets of DNA strands based on the presence or absence
    of the specified bits at positions f, h, and g. It then appends the appropriate sum and carry
    bits to these subsets based on the combination of bits at the specified positions.
    """

  T1, T2 = Extract(T0,y1[f]), Extract(T0,y0[f])
  T3, T4 = Extract(T1,y1[h]), Extract(T1,y0[h])
  T5, T6 = Extract(T2,y1[h]), Extract(T2,y0[h])
  T7, T8 = Extract(T3,y1[g]),  Extract(T3,y0[g])   # finally, T7 contains 111 and T8 contains 110
  T9, T10 = Extract(T4,y1[g]), Extract(T4,y0[g])   # finally, T9 contains 101 and T10 contains 100
  T11, T12 = Extract(T5,y1[g]), Extract(T5,y0[g])  # finally, T11 contains 011 and T12 contains 010
  T13, T14 = Extract(T6,y1[g]), Extract(T6,y0[g])  # finally, T13 contains 001 and T14 contains 000

  T = ['']

  if(Detect(T7)):
    # 1,1,1 gives sum = 1 and carry = 1
    T7 = Append_head(T7, y1[h+1])
    T7 = Append_head(T7, y1[h+2])
    T = Merge(T, T7)

  if(Detect(T8)):
    # 1,1,0 gives sum = 0 and carry = 1
    T8 = Append_head(T8, y0[h+1])
    T8 = Append_head(T8, y1[h+2])
    T = Merge(T, T8)

  if(Detect(T9)):
    # 1,0,1 gives sum = 0 and carry = 1
    T9 = Append_head(T9, y0[h+1])
    T9 = Append_head(T9, y1[h+2])
    T = Merge(T, T9)

  if(Detect(T10)):
    # 1,0,0 gives sum = 1 and carry = 0
    T10 = Append_head(T10, y1[h+1])
    T10 = Append_head(T10, y0[h+2])
    T = Merge(T, T10)

  if(Detect(T11)):
    # 0,1,1 gives sum = 0 and carry = 1
    T11 = Append_head(T11, y0[h+1])
    T11 = Append_head(T11, y1[h+2])
    T = Merge(T, T11)

  if(Detect(T12)):
    # 0,1,0 gives sum = 1 and carry = 0
    T12 = Append_head(T12, y1[h+1])
    T12 = Append_head(T12, y0[h+2])
    T = Merge(T, T12)

  if(Detect(T13)):
    # 0,0,1 gives sum = 1 and carry = 0
    T13 = Append_head(T13, y1[h+1])
    T13 = Append_head(T13, y0[h+2])
    T = Merge(T, T13)

  if(Detect(T14)):
    # 0,0,0 gives sum = 0 and carry = 0
    T14 = Append_head(T14, y0[h+1])
    T14 = Append_head(T14, y0[h+2])
    T = Merge(T, T14)


  return T

In [8]:
def ParallelSumGenerator(T0, n):
  """
    Calculates the sum of each subset and appends sum and carry bit sequences to the corresp. strand.
    Does this by repeatedly and iteratively performing OneBitAdder on all bit positions.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube.
    n: number of elements in L

    Returns:
    list of str: The modified list of DNA strands after performing the binary parallel addition.
    Each strand now contains DNA sequences encoding the sum of its corresponding subset.
    """
  for j in range(n):
    T0 = Append_head(T0, y0[33*(j+1)+16*n])
    for k in range(16):
      T0 = OneBitAdder(T0, 16*j+k,  16*n+33*(j+1)+2*k, 16*n+33*j+2*k+1)
  return T0


In [9]:
def OneBitSubtractor(T0, f, g, h):

  """
    This function takes a list of DNA strands and performs a parallel one-bit subtraction using
    three specified bit positions.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube.
    f (int): The position of the minuend bit.
    g (int): The position of the subtrahend bit.
    h (int): The position of the borrow bit from the previous calculation.

    Returns:
    list of str: The updated list of DNA strands after performing the one-bit subtraction.

    The function works by extracting strands based on the presence or absence of the bits at the
    specified positions and then appending the appropriate difference and borrow bits for each possible
    combination of f, g, and h (1 or 0).
    """

  T1, T2 = Extract(T0,y1[f]), Extract(T0,y0[f])
  T3, T4 = Extract(T1,y1[g]), Extract(T1,y0[g])
  T5, T6 = Extract(T2,y1[g]), Extract(T2,y0[g])
  T7, T8 = Extract(T3,y1[h]),  Extract(T3,y0[h])   # finally, T7 contains 111 and T8 contains 110
  T9, T10 = Extract(T4,y1[h]), Extract(T4,y0[h])   # finally, T9 contains 101 and T10 contains 100
  T11, T12 = Extract(T5,y1[h]), Extract(T5,y0[h])  # finally, T11 contains 011 and T12 contains 010
  T13, T14 = Extract(T6,y1[h]), Extract(T6,y0[h])  # finally, T13 contains 001 and T14 contains 000

  T = ['']

  if(Detect(T7)):
    # 1,1,1 gives difference = 1 and borrow = 1
    T7 = Append_head(T7, y1[h+1])
    T7 = Append_head(T7, y1[h+2])
    T = Merge(T, T7)

  if(Detect(T8)):
    # 1,1,0 gives difference = 0 and borrow = 0
    T8 = Append_head(T8, y0[h+1])
    T8 = Append_head(T8, y0[h+2])
    T = Merge(T, T8)

  if(Detect(T9)):
    # 1,0,1 gives difference = 0 and borrow = 0
    T9 = Append_head(T9, y0[h+1])
    T9 = Append_head(T9, y0[h+2])
    T = Merge(T, T9)

  if(Detect(T10)):
    # 1,0,0 gives difference = 1 and borrow = 0
    T10 = Append_head(T10, y1[h+1])
    T10 = Append_head(T10, y0[h+2])
    T = Merge(T, T10)

  if(Detect(T11)):
    # 0,1,1 gives difference = 0 and borrow = 1
    T11 = Append_head(T11, y0[h+1])
    T11 = Append_head(T11, y1[h+2])
    T = Merge(T, T11)

  if(Detect(T12)):
    # 0,1,0 gives difference = 1 and borrow = 1
    T12 = Append_head(T12, y1[h+1])
    T12 = Append_head(T12, y1[h+2])
    T = Merge(T, T12)

  if(Detect(T13)):
    # 0,0,1 gives difference = 1 and borrow = 1
    T13 = Append_head(T13, y1[h+1])
    T13 = Append_head(T13, y1[h+2])
    T = Merge(T, T13)

  if(Detect(T14)):
    # 0,0,0 gives difference = 0 and borrow = 0
    T14 = Append_head(T14, y0[h+1])
    T14 = Append_head(T14, y0[h+2])
    T = Merge(T, T14)


  return T

In [10]:
def ParallelSubtractor(T0, n):
  """
    Subtracts P from the target number t, where P is any subset sum. It does this for all 2^(n/2) subset sums taken from the second half of input list L.
    Appends the difference and borrow bits to the corresponding strands.

    Parameters:
    T0 (list of str): A list of DNA strands in a test tube, representing the current state of the
                      computational process.
    n: number of elements in L

    Returns:
    list of str: The updated list of DNA strands after performing the binary parallel subtraction.

    Iterates through 16 bit positions. For each position, it uses the OneBitSubtractor
    function, passing three specific bit positions: one from the first set (i + 180), another from the
    second set (148 + 2*i), and a third for the borrow bit from a previous calculation (196 + 2*i).
    Function subtracts the second set of bits from the first set across all 16 positions.
    """
  for i in range(16):
    T0 = OneBitSubtractor(T0, i+16*n+33*(n+1), 16*n+33*n+1+2*i, 16*n+33*(n)+49+2*i)
  return T0

In [11]:
def ParallelSearcher(T1, T2, n):
  """
  Performs bitwise comparison of the bits from the sums in T1 and differences in T2.
  Parameters:
  T1 (list of str): Contains sums of subsets generated from the first half of L
  T2 (list of str): Contains differences between target t and sums of subsets generated from the second half of L
  n: number of elements in L
  """
  for i in range(16):
    T3, T4 =  Extract(T1,y1[16*n+33*n+1+2*i]), Extract(T1,y0[16*n+33*n+1+2*i])
    T5, T6 = Extract(T2,y1[16*n+33*n+1+2*i+49]), Extract(T2,y0[16*n+33*n+1+2*i+49])
    temp = ''.join([ random.choice(list(pairings.keys())) for _ in range(pow(2,i+1)) ])
    T3, T5 = Append_head(T3, temp), Append_head(T5, temp)
    T1, T2 = Merge(T3,T4), Merge(T5,T6)
  T1 = Merge(T1, T2)
  return T1

In [12]:
def find_pairs_with_same_length(strings):
    """
    Parameters:
    strings (list of str): A list of strings among which pairs with a specific length difference (784)
                           are to be identified.

    Returns:
    list of tuples: A list of tuples, where each tuple contains the indices of a pair of strings
                    in the original list that satisfy the length difference condition.
    """
    pairs = []
    for i in range(len(strings)):
        for j in range(i + 1, len(strings)):
            if (len(strings[i])+784) == len(strings[j]):
                pairs.append((i, j))
    for i in range (len(pairs)):
      for j in range(V):
        if(x[j][0] in T1[pairs[i][0]]):
          print(numbers[j])
        if(x[j][0] in T1[pairs[i][1]]):
          print(numbers[j])
    return pairs

In [13]:
n = int(input('Enter the number of elements in L: '))

Enter the number of elements in L: 7


In [14]:
def binary_reversed(number):
    """Converts a number to a reversed 16-bit binary representation."""
    return list(map(int, format(number, '016b')[::-1]))

# Taking input from the user
input_numbers = input("Enter a list of numbers separated by space: ")
numbers = list(map(int, input_numbers.split()))
print(f'{numbers}')
# Converting each number to a reversed 16-bit binary representation
binary_2d_list = [binary_reversed(num) for num in numbers]

c = [[] for _ in range(n)]
# contains the binary representation of each element in L
for i in range(n):
    c[i] = [''] * 16   # initialising

for i in range(16):
    for j in range(n):
        c[j][i] = y0[16 * j + i]    # initialising all bits to 0

for i in range(n):
    for j in range(16):
        if(binary_2d_list[i][j]):
          c[i][j] = y1[j+16*i]


Enter a list of numbers separated by space: 1 2 3 100 200 4 5
[1, 2, 3, 100, 200, 4, 5]


In [15]:
  elements = ['18', '75', '9']   # Elements in our input list L
  V = n

  element_strands = [ ''.join([ random.choice(list(pairings.keys())) for _ in range(K) ]) for _ in range(V) ]
  print(f"Element Strands Generated: {element_strands}")
  complements = [ find_complement(v) for v in element_strands]
  print(f"Complement Strands Generated: {complements}")

  # Pairing strands and complements
  x = list(zip(element_strands, complements))
  print(f'{x}')


Element Strands Generated: ['TACATGACCAACGCAC', 'GACTTAGACGAGTGGC', 'ATCCTTGTCCGGGAGG', 'GCGAGAGGGGTGACAT', 'TAAAGATAATTGCGCT', 'GAGACCGACAGTCGAG', 'TACGGGCGCATTTAAA']
Complement Strands Generated: ['ATGTACTGGTTGCGTG', 'CTGAATCTGCTCACCG', 'TAGGAACAGGCCCTCC', 'CGCTCTCCCCACTGTA', 'ATTTCTATTAACGCGA', 'CTCTGGCTGTCAGCTC', 'ATGCCCGCGTAAATTT']
[('TACATGACCAACGCAC', 'ATGTACTGGTTGCGTG'), ('GACTTAGACGAGTGGC', 'CTGAATCTGCTCACCG'), ('ATCCTTGTCCGGGAGG', 'TAGGAACAGGCCCTCC'), ('GCGAGAGGGGTGACAT', 'CGCTCTCCCCACTGTA'), ('TAAAGATAATTGCGCT', 'ATTTCTATTAACGCGA'), ('GAGACCGACAGTCGAG', 'CTCTGGCTGTCAGCTC'), ('TACGGGCGCATTTAAA', 'ATGCCCGCGTAAATTT')]


In [ ]:
# Running the Init function
T1 = Init(0, (int)(V/2))
# T0 now contains all possible subsets of the elements
for strand in T1:
    print(strand)
T2 = Init((int)(V/2), V)
# T0 now contains all possible subsets of the elements
for strand in T2:
    print(strand)

In [17]:
for i in range((int)(V/2), V):
  T1 = Append(T1, x[i][1])
for i in range((int)(V/2)):
  T2 = Append_head(T2, x[i][1])

In [ ]:
for strand in T1:
    print(strand)
for strand in T2:
    print(strand)

In [19]:
t=[]  # list containing 16 bits of the target number t
p=16*n+33*(n+1)   # starting index to pick up bits fro y0 and y1
i=0  # starting bit position to initialise the bits of t
num = int(input("Enter the target number: "))
binary_num = bin(num)[2:]
# Ensuring that the binary representation has exactly 16 bits by padding zeros to the left
binary_num = binary_num.zfill(16)

for bit in reversed(binary_num):
  if(bit=='1'):
    t.append(y1[p+i])
  else:
    t.append(y0[p+i])
  i+=1
t0 = y0[p+16]
print(f'{t}') # t now contains 16 DNA sequences, one for each bit

Enter the target number: 8
['TCAGTGTGAATGTCTA', 'GTCCAGCACATTCAGA', 'TAGCCCGTAGATTGTT', 'GTTGCAGACACAGTCG', 'TTCGACTAGGCCCTAA', 'TCCACTAATCGTCGCA', 'TGCACAAACGTCACGG', 'TCAGTTTTAAAGCGAA', 'CGCATTAGAAAACGAC', 'GGGCCGCTTAAGAACC', 'TGGATTGGCGCGAGCT', 'TTCCCTACTGGTATCG', 'TACAAATACGACTTCT', 'ACAGTCGTCGTTTTCT', 'ACGTTCTGCCAGGACC', 'TAAGTTCGATCCTATA']


In [20]:
T1 = Value(T1, n)
T1 = InitialSum(T1, n)
T2 = Value(T2, n)
T2 = InitialSum(T2, n)
T1 = ParallelSumGenerator(T1, n)
T2 = ParallelSumGenerator(T2, n)
for i in range(16):
    T2 = Append_head(T2, t[i])
T2 = Append_head(T2, t0) # initialising the first input borrow bit to 0
T2 = ParallelSubtractor(T2, n)

In [21]:
T1 = ParallelSearcher(T1, T2, n)

In [22]:
find_pairs_with_same_length(T1)

1
3
4
3
5
1
2
5


[(2, 22), (3, 23), (4, 23)]